In [1]:
import bz2
import re
from collections import defaultdict
import sys

In [ ]:
abst = {}
with bz2.BZ2File('../long_abstracts_en.tql.bz2', 'r') as f:
        for i, line in enumerate(f):
            if i == 0:
                continue
            split = line.split(b' ', 2)
            v = split[2].decode('utf-8').split('@')[0]
            k = split[0].decode('utf-8').split('/')[-1][:-1]
            abst[k] = v

In [ ]:
RE_SCHEMA = re.compile(r'(.*?)\s.*schema\.org\/(.*)')
ent2type = {}
with open('../dbpedia/out/instanceTypes.tsv', 'r') as f:
    for i, line  in enumerate(f):
        m = RE_SCHEMA.match(line)
        if m:
            ent2type[m.group(1)] = m.group(2)

In [ ]:
with open('abst.tsv', 'w') as f:
    for k, v in abst.items():
        f.write('{}\t{}\n'.format(k, v))

In [ ]:
with open('ent2type.tsv', 'w') as f:
    for k, v in ent2type.items():
        f.write('{}\t{}\n'.format(k, v))

In [10]:
abst = {}
with open('../data_ner/abst.tsv', 'r') as f:
    for line in f:
        try:
            k, v = line.split('\t', 2)
        except ValueError:
            print(line)
        abst[k] = v

In [11]:
len(abst)

4932176

In [ ]:
regp = re.compile(r'.*resource\/(.*?)\?.*(phrase|word)&char\=(\d+\,\d+)\>.*taIdentRef.*resource\/(.*?)\>')
links = defaultdict(list)
counter = 0
with bz2.BZ2File('../nif_text_links_en.tql.bz2') as f:
    for i, line in enumerate(f):
        if i == 0:
            continue
        text = line.decode('utf-8')
        m = regp.match(text)
        if m is None:
            continue
        
        p_id = m.group(1)
        if p_id not in abst:
            continue
            
        offsets = m.group(3)
        entity = m.group(4)

        end = int(offsets.split(',')[1])
        v = offsets + ',' + entity
        if v not in links[p_id] and end < len(abst[p_id]):
            links[p_id].append(v)
        counter += 1
        if counter % 10**6 == 0:
            print(links[p_id])
            print(i)

['96,116,Muslim_world', '220,225,Mecca', '230,236,Medina', '291,305,Historical_powers#Arab_Caliphates', '320,326,Al-Andalus', '346,357,Indus_river', '407,415,Umayyads', '449,455,Caliphate_of_Córdoba', '458,466,Abbasids', '468,476,Fatimids', '482,489,Mamluk', '547,567,Islamic_Golden_Age', '609,616,Science_in_the_medieval_Islamic_world', '638,649,Astronomy_in_the_medieval_Islamic_world', '651,665,Mathematics_in_medieval_Islam', '667,674,Medicine_in_the_medieval_Islamic_world', '679,691,Islamic_philosophy', '703,722,Golden_Age_of_Islam', '993,1009,Mongol_invasions', '1066,1077,Black_Death', '1186,1205,Early_Modern_period', '1211,1219,Ottoman_Empire', '1225,1233,Safavid_dynasty', '1243,1250,Mughal_Empire', '1335,1347,Muslim_world', '1399,1411,Great_power']
7398837
['15,31,American_English', '43,58,British_English', '79,85,Potato', '100,110,Deep_frying', '197,202,Snack', '204,213,Side_dish', '218,227,Appetizer', '260,266,Edible_salt', '357,362,Herbs', '364,370,Spice', '372,379,Cheese', '385

In [ ]:
len(links)

In [ ]:
for i, (k,v) in enumerate(dict(links).items()):
    if i == 1000:
        break
    print(k, v)

In [ ]:
mentions = links['Animalia_(book)']
print(mentions)
for mention in mentions:
    start, end, _ = mention.split(',')
    start, end = int(start), int(end)
    print(abst['Animalia_(book)'][start:end])

In [ ]:
for page_id, offsets in links.items():
    for offset in offsets:
        split = offset.split(',')
        start, end = int(split[0]) + 1, int(split[1]) + 1
        text = abst[page_id]
        if end < len(text):
            print(abst[page_id][start:end])